In [62]:
from IPython.display import Latex
import math
import numpy as np
import sympy
from fractions import Fraction

cheems_1 = [('C',1),('H',1),([('C',1),('H',2)],7),('C',1),('O',2),('C',1),('H',3)]
chem_2 = [('Al', 2), ([('S', 1), ('O', 4)], 3)]

def chemical_formula(chem, pre_str_chem= ''):
  str_chem = pre_str_chem
  for i, sub_chem in enumerate(chem):
    if type(sub_chem[0]) is list:
      str_chem = str_chem + '('
      str_chem = chemical_formula(sub_chem[0], pre_str_chem=str_chem)
      str_chem = str_chem + ')_' + str(sub_chem[1])
    else:
      if sub_chem[1] > 1:
        str_chem = str_chem + r'\text{' + str(sub_chem[0]) + r'}_' + str(sub_chem[1])
      else:
                str_chem = str_chem + r'\text{' + str(sub_chem[0]) + r'}'
  return str_chem 



In [63]:
def chemical_element(chem):
  chemical_list = []  
  for sub_chem in chem:
    if type(sub_chem[0]) is list:
      chemical_list +=chemical_element(sub_chem[0])
    else:
      if sub_chem[0] not in chemical_list:
        chemical_list.append(sub_chem[0])
  return chemical_list


In [64]:
def count_element(result, chem, integer, element_list):
  for sub_chem in chem:
      if type(sub_chem[0]) is list:
        count_element(result, sub_chem[0], sub_chem[1],element_list)
        
      else:
          for i, element in enumerate(element_list):
              if sub_chem[0] == element:
                  result[i] = result[i] + sub_chem[1]*integer;
  return result


In [65]:
def print_fraction(weight):
  result = r''
  if weight[1] == 1:
    result =  str(weight[0])
  else:
    result =  r'\frac{' + str(weight[0]) + r'}{' + str(weight[1]) + r'}'
  return result

In [66]:
def result_string(lhs,rhs,weight_formula):
  result_string = r''
  #result_string = ''
  temp = 0
  for i,chem in enumerate(lhs):
    if i != 0:
      result_string += r' + '
    result_string += str(print_fraction(float_to_fraction(weight_formula[i]))) + ' ' + chemical_formula(chem)
    temp = i
  result_string += r' \to '
  for i,chem in enumerate(rhs):
    if i != 0:
      result_string += ' + '
    result_string += str(print_fraction(float_to_fraction(weight_formula[i+temp+1]))) + ' ' + chemical_formula(chem)
  return result_string

In [67]:
def balance_chemical_equation(lhs, rhs):
  "lhs là danh sách các phân tử tham gia phản ứng, rhs là danh sách các phân tử sản phẩm"
  all_chem = lhs
  element_list = []

  for chem in lhs:
    element_list += chemical_element(chem)
  element_list = list(dict.fromkeys(element_list))

  lhs_array = np.array([])
  rhs_array = np.array([])
  
  for chem in lhs:
    num_array = np.zeros(len(element_list))
    count_element(num_array,chem,1, element_list)
    lhs_array = np.concatenate((lhs_array,num_array))
  
  for chem in rhs:
    num_array = np.zeros(len(element_list))
    count_element(num_array,chem,1, element_list)
    rhs_array = np.concatenate((rhs_array,num_array))
  
  lhs_array = lhs_array.reshape(len(lhs),len(element_list))
  lhs_array = lhs_array.T
  rhs_array = rhs_array.reshape(len(rhs),len(element_list))
  rhs_array = rhs_array.T
  rhs_array *= -1
  #rhs_array = np.expand_dims(rhs_array,axis=0)
  
  Array = np.concatenate((lhs_array, rhs_array),axis=1)
  b = np.zeros(len(element_list))
  b = b.reshape(len(element_list),1)
  Array = np.concatenate((Array,b),axis=1)
  Array = sympy.Matrix(Array)

  symbol = [sympy.symbols(f"x{i}") for i in range (1,len(lhs)+len(rhs)+1) ]
  weight_formula = sympy.linsolve(Array, symbol)
  weight_formula = weight_formula.subs(symbol[len(lhs)+len(rhs)-1],1)
  weight_formula = list(weight_formula)
  result = result_string(lhs,rhs,weight_formula[0])

  return result

In [68]:
def float_to_fraction (x, error=0.000001):
    n = int(math.floor(x))
    x -= n
    if x < error:
        return (n, 1)
    elif 1 - error < x:
        return (n+1, 1)

    # The lower fraction is 0/1
    lower_n = 0
    lower_d = 1
    # The upper fraction is 1/1
    upper_n = 1
    upper_d = 1
    while True:
        # The middle fraction is (lower_n + upper_n) / (lower_d + upper_d)
        middle_n = lower_n + upper_n
        middle_d = lower_d + upper_d
        # If x + error < middle
        if middle_d * (x + error) < middle_n:
            # middle is our new upper
            upper_n = middle_n
            upper_d = middle_d
        # Else If middle < x - error
        elif middle_n < (x - error) * middle_d:
            # middle is our new lower
            lower_n = middle_n
            lower_d = middle_d
        # Else middle is our best fraction
        else:
            return (n * middle_d + middle_n, middle_d)

In [69]:
Al_OH3 = [("Al", 1), ([("O", 1), ("H", 1)], 3)]
H2_SO4 = [("H", 2), ("S", 1), ("O", 4)]
Al2_SO43 = [("Al", 2), ([("S", 1), ("O", 4)], 3)]
H2O = [("H", 2), ("O", 1)]
H2 = [("H", 2)]
O2 = [("O", 2)]
KMnO4 = [('K',1),('Mn',1),('O',4)]
MnO2 = [('Mn',1),('O',2)]
K2MnO4 = [('K',2),('Mn',1),('O',4)]

Al = [('Al',1)]
Al_NO33 = [("Al", 1), ([("N", 1), ("O", 3)], 3)]
H_NO3 = [("H", 1), ("N", 1), ("O", 3)]
NO2 = [('N',1),('O',2)]


equation = balance_chemical_equation([Al, H_NO3],[Al_NO33, NO2, H2O])
# sẽ được như
#equation = r"2\text{Al}(\text{O}\text{H})_3 + 3\text{H}_2\text{S}\text{O}_4 \to \text{Al}_2(\text{S}\text{O}_4)_3 + 6\text{H}_2\text{O}"

print(equation)
Latex("$$" + equation + "$$")

\frac{1}{3} \text{Al} + 2 \text{H}\text{N}\text{O}_3 \to \frac{1}{3} \text{Al}(\text{N}\text{O}_3)_3 + 1 \text{N}\text{O}_2 + 1 \text{H}_2\text{O}


<IPython.core.display.Latex object>